In [ ]:
from tensorflow import keras
import numpy as np


**Q1**

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
train_images.shape[2]

28

**Q2**

In [ ]:
def sigmoid(layer,d=False):
    if(d==True):
        return sigmoid(layer)*(1-sigmoid(layer))
    return 1.0/(1.0+np.exp(-1.0*layer))

def softmax(layer):
    #new_layer=(layer-np.min(layer))/(np.max(layer)-np.min(layer))
    return np.exp(layer)/np.sum(np.exp(layer))

def d_sigmoid(layer):
    return sigmoid(layer)*(1-sigmoid(layer))

def cross_entropy(true_output,output):
    return -1.0*np.sum(true_output * np.log(output))

def tanh(layer,d=False):
    if(d==True):
        return 1-(np.exp(layer)-np.exp(-layer))/(np.exp(layer)+np.exp(-layer))**2
    return (np.exp(layer)-np.exp(-layer))/(np.exp(layer)+np.exp(-layer))

def d_tanh(layer):
    return 1-(np.exp(layer)-np.exp(-layer))/(np.exp(layer)+np.exp(-layer))**2

def relu(layer,d=False):
    if(d==True):
      return 1. * (layer > 0)
    return layer * (layer > 0)

def d_relu(layer):
    return 1. * (layer > 0)

In [ ]:
input_size=28*28
output_size=10
num_hidden_layers=2
num_neurons_in_hidden_layer=16
pre_activation_layer={}
activation_layer={}
neurons_list=[]
weights={}
bias={}
input_layer=np.random.rand(28*28,1)#to be defined n*1
# output_layer=np.zeros((output_labels,1))

neurons_list.append(input_size)
for i in range(num_hidden_layers):
  neurons_list.append(num_neurons_in_hidden_layer)
neurons_list.append(output_size)

#hidden_layer[0]=input_layer
# for i in range(num_hidden_layers):
#   hidden_layer.append(np.zeros(1,num_neurons_in_hidden_layer))
# hidden_layer.append(output_layer)

activation_layer[0]=input_layer
# for i in range(num_hidden_layers):
#   activation_layer.append(np.zeros(1,num_neurons_in_hidden_layer))
# activation_layer.append(output_layer)

for i in range(len(neurons_list)-1):
  weights[i+1]=np.random.rand(neurons_list[i+1],neurons_list[i])

for i in range(len(neurons_list)-1):
  bias[i+1]=np.random.rand(neurons_list[i+1],1)


def forward_propogation():
  for i in range(1,num_hidden_layers+1):
    pre_activation_layer[i]=bias[i]+np.matmul(weights[i],activation_layer[i-1])
    activation_layer[i]=sigmoid(pre_activation_layer[i])
  pre_activation_layer[num_hidden_layers+1]=bias[num_hidden_layers+1]+np.matmul(weights[num_hidden_layers+1],activation_layer[num_hidden_layers])
  activation_layer[num_hidden_layers+1]=softmax(pre_activation_layer[num_hidden_layers+1])

forward_propogation()
  

In [ ]:
class FeedFowardNeuralNetwork:
  def __init__(self,num_hidden_layers,num_neurons_in_hidden_layer):
    self.num_hidden_layers=num_hidden_layers
    self.num_neurons_in_hidden_layer=num_neurons_in_hidden_layer
    
  def weight_initialization(self):
    self.pre_activation_layer={}
    self.activation_layer={}
    self.weights={}
    self.bias={}
    self.neurons_list=[self.input_size]

    for i in range(self.num_hidden_layers):
      self.neurons_list.append(self.num_neurons_in_hidden_layer)
    self.neurons_list.append(self.output_size)

    self.activation_layer[0]=self.input_layer

    for i in range(len(self.neurons_list)-1):
      self.weights[i+1]=np.random.rand(self.neurons_list[i+1],self.neurons_list[i])

    for i in range(len(self.neurons_list)-1):
      self.bias[i+1]=np.random.rand(self.neurons_list[i+1],1)
  
  def forward_propogation(self):
    for i in range(1,self.num_hidden_layers+1):
      self.pre_activation_layer[i]=self.bias[i]+np.matmul(self.weights[i],self.activation_layer[i-1])
      self.activation_layer[i]=sigmoid(self.pre_activation_layer[i])
    self.pre_activation_layer[self.num_hidden_layers+1]=self.bias[self.num_hidden_layers+1]+np.matmul(self.weights[self.num_hidden_layers+1],self.activation_layer[self.num_hidden_layers])
    self.activation_layer[self.num_hidden_layers+1]=softmax(self.pre_activation_layer[self.num_hidden_layers+1])    
    return self.activation_layer[self.num_hidden_layers+1]

  def run(self,X,Y):
    self.input_layer=X
    self.input_size=len(X)
    self.output_size=Y
    self.weight_initialization()
    return self.forward_propogation()


nn=FeedFowardNeuralNetwork(1,128)
print(nn.run(train_images[0].reshape((-1,1)),10))# Converting to column vector

